# Imports

In [1]:
import re
import string
import unicodedata
import pickle
from collections import Counter
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from scipy.sparse import hstack

import contractions

import nltk
from nltk.corpus import stopwords

import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

from sklearn import (
    linear_model,
    feature_extraction,
    model_selection,
    naive_bayes,
    metrics,
    ensemble,
)
from sklearn.preprocessing import normalize


import pyLDAvis
import pyLDAvis.gensim_models

import hypopt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit




C:\Users\yagne\anaconda3\envs\common\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\yagne\anaconda3\envs\common\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


# Data

In [2]:
data = pd.read_csv("data/redo_clean.csv")
data["cleaned_text2"] = data["cleaned_text2"].apply(lambda x: x.split(" "))

train = data.loc[data["split"] == "train"]
val = data.loc[data["split"] == "val"]
test = data.loc[data["split"] == "test"]
print(train.shape, val.shape, test.shape)

(83818, 15) (9314, 15) (10348, 15)


# Vectorize

In [3]:
id2word = corpora.Dictionary(train["cleaned_text2"])
train_corpus = [id2word.doc2bow(text) for text in train["cleaned_text2"]]
val_corpus = [id2word.doc2bow(text) for text in val["cleaned_text2"]]
test_corpus = [id2word.doc2bow(text) for text in test["cleaned_text2"]]
print(train_corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]


In [4]:
topics = 20
alpha = 1.0

In [5]:
lda_model = gensim.models.ldamulticore.LdaMulticore(
            corpus=train_corpus,
            id2word=id2word,
            num_topics=topics, 
            random_state=100,
            workers=5,
            passes=3,
            alpha=alpha,
            per_word_topics=True
        )

In [6]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, id2word, sort_topics=False)
vis

C:\Users\yagne\anaconda3\envs\common\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.028277 -0.003417       1        1   4.151010
1     -0.014264 -0.007658       2        1   3.793052
2     -0.011232  0.005476       3        1   4.376471
3      0.000766 -0.003221       4        1   3.320233
4     -0.092644 -0.080110       5        1   7.098479
5      0.006720 -0.002308       6        1   3.313485
6     -0.000948 -0.017159       7        1   4.275620
7      0.010981  0.003099       8        1   3.447285
8      0.010905 -0.007404       9        1   3.945295
9     -0.034514  0.008103      10        1   4.574018
10     0.082968 -0.012424      11        1   7.745063
11     0.027613  0.010016      12        1   5.214840
12    -0.033998  0.018237      13        1   4.713592
13     0.003394 -0.006468      14        1   3.606686
14     0.008179 -0.032679      15        1   4.772195
15    -0.019528 -0.001227      16        1   3.545704
16     0.019321  0.037722      17        1   4.723445
17     0.038263  0.026320      18        1   5.782684
18     0.113449 -0.008277      19        1  11.874305
19    -0.087154  0.073379      20        1   5.726536, topic_info=        Term          Freq         Total Category  logprob  loglift
8       game  10262.000000  10262.000000  Default  30.0000  30.0000
710      eat   4856.000000   4856.000000  Default  29.0000  29.0000
2304    play   8553.000000   8553.000000  Default  28.0000  28.0000
10316     de    949.000000    949.000000  Default  27.0000  27.0000
1566     pay   3770.000000   3770.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1439   issue    515.479774   4436.775572  Topic20  -6.2898   0.7075
25      time    594.102851  14425.970856  Topic20  -6.1478  -0.3297
39      help    491.988242   8678.605512  Topic20  -6.3364  -0.0101
775    thank    466.008671   4334.820716  Topic20  -6.3907   0.6298
1389    work    441.369983   4969.929146  Topic20  -6.4450   0.4388

[1867 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
25154     13  0.925531     aber
24252      7  0.034194  abraham
24252     15  0.854839  abraham
24252     20  0.034194  abraham
571        1  0.029498  account
...      ...       ...      ...
4561      17  0.386761   zoloft
4561      18  0.047553   zoloft
4561      19  0.139488   zoloft
4561      20  0.015851   zoloft
4748      13  0.946499       zu

[14250 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [7]:
topic_terms = []
for i in range(topics):
    topic_terms.append({
        "topic": i,
        "terms": ", ".join([id2word.id2token[x[0]] for x in lda_model.get_topic_terms(i, 20)])
    })

topic_terms = pd.DataFrame(topic_terms)
topic_terms["marginal_topic_distribution"] = vis.to_dict()['mdsDat']['Freq']


In [16]:
topic_terms.sort_values("marginal_topic_distribution", ascending=False)

,topic,terms,marginal_topic_distribution,coef
18,18,"friend, feel, feel_like, relationship, people,...",11.874305,1.140667
10,10,"she_is, time, girl, say, feel, think, see, my_...",7.745063,-0.180723
4,4,"world, good, character, post, book, people, lo...",7.098479,-1.765496
17,17,"pay, job, money, time, trip, help, month, hour...",5.782684,0.051897
19,19,"game, use, music, pc, run, build, buy, case, p...",5.726536,-0.982262
11,11,"feel, time, eat, good, trip_report, sleep, peo...",5.214840,0.560200
14,14,"god, people, life, feel, time, say, think, mak...",4.772195,-0.056456
16,16,"feel, car, time, take, help, use, start, year,...",4.723445,0.783375
12,12,"game, play, good, take, think, feel, pain, lov...",4.713592,-0.618864
9,9,"use, people, good, time, help, character, thin...",4.574018,-0.473890


In [9]:
X_train = lda_model.get_document_topics(train_corpus, minimum_probability=0.01, per_word_topics=False)
X_train = gensim.matutils.corpus2csc(X_train).transpose()

X_val = lda_model.get_document_topics(val_corpus, minimum_probability=0.01, per_word_topics=False)
X_val = gensim.matutils.corpus2csc(X_val).transpose()

X_test = lda_model.get_document_topics(test_corpus, minimum_probability=0.01, per_word_topics=False)
X_test = gensim.matutils.corpus2csc(X_test).transpose()

In [10]:

X_train = X_train.toarray()
X_train = (X_train - X_train.min(0)) / X_train.ptp(0) 
X_val = (X_val.toarray() - X_train.min(0)) / X_train.ptp(0) 
X_test = (X_test.toarray() - X_train.min(0)) / X_train.ptp(0) 

In [11]:
y_train = train["depression"].to_numpy()
y_val = val["depression"].to_numpy()
y_test = test["depression"].to_numpy()

# Model

In [13]:
X_train_val = np.vstack((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))
split_index = [-1]*X_train.shape[0] + [0]*X_val.shape[0]
X_train_val.shape

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

estimator = linear_model.LogisticRegression()
param_grid = [
  {"penalty":["elasticnet"], "l1_ratio":[0,0.1,0.5,0.9,1], "C":[0,0.1,0.5,1,2], "solver":["saga"], "max_iter": [200]}
 ]

# Use PredefinedSplit in GridSearchCV
clf = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid,
                   scoring="f1"
                   )

clf.fit(X_train_val, y_train_val)

C:\Users\yagne\anaconda3\envs\common\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\yagne\anaconda3\envs\common\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\yagne\anaconda3\envs\common\lib\site-packages\sklearn\linear_model\_logistic.py", line 1589, in fit
    fold_coefs_ = Parallel(
  File "C:\Users\yagne\anaconda3\envs\common\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(itera

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=LogisticRegression(),
             param_grid=[{'C': [0, 0.1, 0.5, 1, 2],
                          'l1_ratio': [0, 0.1, 0.5, 0.9, 1], 'max_iter': [200],
                          'penalty': ['elasticnet'], 'solver': ['saga']}],
             scoring='f1')

In [14]:
y_pred = clf.best_estimator_.predict(X_test)
print((metrics.classification_report(y_test, y_pred)))

              precision    recall  f1-score   support

         0.0       0.59      0.77      0.67      5509
         1.0       0.60      0.39      0.47      4839

    accuracy                           0.59     10348
   macro avg       0.59      0.58      0.57     10348
weighted avg       0.59      0.59      0.58     10348



In [15]:
topic_terms["coef"] = clf.best_estimator_.coef_[0,:]
topic_terms.to_csv("topic_results.csv",index=False)